# Convert  ".root" data into ".pkl"

In [1]:
%pylab inline

import uproot
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Loading data and MC samples

In [2]:
df_event=uproot.open('../../../HypertritonAnalysis/Trees/HyperTritonTree_18r.root')['_custom']['fTreeV0'].pandas.df('fCent')

In [3]:
Nev_1040=len(df_event.query('10<fCent<=40'))

In [5]:
df_MC = uproot.open('../../../HypertritonAnalysis//Derived_Trees/HyperTree_MC_Signal.root')['HyperTree_MC_Signal'].pandas.df()

In [6]:
df_data = uproot.open('../../../HypertritonAnalysis/Derived_Trees/HyperTree_Data.root')['HyperTree_Data'].pandas.df()

## Calculate pre selection efficiency

In [7]:
cuts_presel='ProngsDCA<1.6 and He3ProngPvDCA>0.01 and He3ProngPvDCA>0.01 and V0CosPA>0.99 and Centrality<10'

In [8]:
eff_v0=[0.23,0.32,0.42,0.51]

In [9]:
eff_23=eff_v0[0]*len(df_MC.query(cuts_presel+' and 2<V0pt<3'))/len(df_MC.query('2<V0pt<3 and Centrality<10'))

In [10]:
eff_34=eff_v0[1]*len(df_MC.query(cuts_presel+' and 3<V0pt<4'))/len(df_MC.query('3<V0pt<4 and Centrality<10'))

In [11]:
eff_45=eff_v0[2]*len(df_MC.query(cuts_presel+' and 4<V0pt<5'))/len(df_MC.query('4<V0pt<5 and Centrality<10'))

In [12]:
eff_59=eff_v0[3]*len(df_MC.query(cuts_presel+' and 5<V0pt<9'))/len(df_MC.query('5<V0pt<9 and Centrality<10'))

In [13]:
eff_presel=[eff_23,eff_34,eff_45,eff_59]

In [15]:
np.save("../PreSelEfficiency/eff_presel_0_10",eff_presel)

## ML dataframe 10-40

In [16]:
df_MC=df_MC.query(cuts_presel)

In [17]:
df_Bkg = df_data.query('(InvMass<2.98 or InvMass>3.005) and V0pt<=10')
df_Sig_data=df_data.query('2.960<InvMass<3.050 and V0pt<=10')

In [18]:
df_Bkg['y']=0
df_MC['y']=1
df_ML= pd.concat([df_MC,df_Bkg.query('Centrality<10')])
df_ML.to_pickle('../../../HypertritonAnalysis/Dataframes/df_ML.pkl')

/home/fmazzasc/.local_pip/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
df_Sig_data.query('Centrality<10').to_pickle('../../../HypertritonAnalysis/Dataframes/df_Sig_data.pkl')